## Microsoft Learn Project
* This Notebook has been powered by and inspired by:
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>

</div>


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')


Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [ ]:
import os
import re
import subprocess
import time
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import torch
from google.colab import userdata
from typing import Optional
from datasets import Dataset

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)



#### Setup Configs

In [ ]:
class Config:
  device = 'gpu' #change this to cpu and change runtime to cpu but inference is extremely slow and will take approximately 2 days max, gpu inference takes only 2hrs
  bits = 8
  folder_path = '/content/gdrive/MyDrive/Microsoft_Learn_Location_Mention-Recognition-Challenge/data'#folder with the tg booklets
  train_filepath = f'{folder_path}/Train_1.csv'
  test_filepath = f'{folder_path}/Test.csv'
  model_name = "unsloth/gemma-2-9b-bnb-4bit"
  max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
  dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
  load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.



In [ ]:
train = pd.read_csv(Config.train_filepath)
test = pd.read_csv(Config.test_filepath)
print(train.shape, test.shape)
display(train.head(), test.head())

(73072, 3) (2942, 2)


,tweet_id,text,location
0,ID_1001136212718088192,NaN,EllicottCity
1,ID_1001136696589631488,"Flash floods struck a Maryland city on Sunday, washing out streets and tossing cars like bath toys.",Maryland
2,ID_1001136950345109504,State of emergency declared for Maryland flooding: via @YouTube,Maryland
3,ID_1001137334056833024,"Other parts of Maryland also saw significant damage from Sundays storms including this Baltimore city neighborhood, #Dundalk and #Catonsville. Rain totals spanned from 1 to 10 inches across Maryland: #ECFlood",Baltimore Maryland
4,ID_1001138374923579392,"Catastrophic Flooding Slams Ellicott City, Maryland; Water Rescues Reported - The Weather Channel via @GoogleNews",Ellicott City Maryland


,tweet_id,text
0,ID_1001154804658286592,"What is happening to the infrastructure in New England? It isnt global warming, its misappropriated funds being abused that shouldve been used maintaining their infrastructure that couldve protected them from floods! Like New Orleans. Their mayor went to ὄ7#Maryland #floods"
1,ID_1001155505459486720,"SOLDER MISSING IN FLOOD.. PRAY FOR EDDISON HERMOND! PRAY FOR ELLICOTT CITY, MARYLAND! #PrayForEddisonHermond #PrayForEllicottCity"
2,ID_1001155756371136512,"RT @TIME: Police searching for missing person after devastating 1,000-year flood in Ellicott City, Maryland"
3,ID_1001159445194399744,"Flash Flood Tears Through Maryland Town For Second Time In Two Years Less than two years after what had been called a once in a 1,000 years” flood in 2016, Ellicott City, Md., sees its historic downtown ravaged anew. One man remains missing. from Flas"
4,ID_1001164907587538944,Ellicott City #FLOODING Pictures: Maryland Governor Declares State of Emergency After Severe Flash #FLOODS


In [ ]:
train.dropna(inplace=True)
train.shape

(11849, 3)

In [ ]:
train[['text', 'location']].tail(10)

,text,location
73057,RT @fronterasdesk: Rescue teams in Mexico City have recovered the last body known to be caught under rubble after the earthquake. /,Mexico City
73060,Cuban doctors treat Mexico’s earthquake victims,Mexico
73062,RT @DJmag: ὄF @Ultra Mexico to donate ticket sales to Mexico City earthquake relief fund ὄF ὄ9,Mexico Mexico City
73064,Mexico Earthquake Relief Font Bundle,Mexico
73065,RT @nytimesworld: Buildings that survived the 1985 Mexico City earthquake seemed resilient. Then they fell in the recent quake.,Mexico City
73066,"Mexico City: at least a thousand buildings damaged by the 19 September earthquake must be bulldozed, thousands others need repair",Mexico City
73068,Rescue workers recover the body of the last person known to be missing after the Mexico City earthquake,Mexico City
73069,Donate from Facebook to Mexico Earthquake Relief ὉAᾕ1❤,Mexico
73070,We are helping our clients in Mexico recover from the earthquake. Were proud to help rebuild the community weve been serving since 1945.,Mexico
73071,Thanks @RFC_Charity @UNICEFMexico for the support! Like them you can donate & help our fellow people in Mexico ἟2἟D⬇️,Mexico


#### Modelling
* Load model and add LoRA adapters so we only need to update 1 to 10% of all parameters!
* Install model dependencies ⁉
  * If prompted to install dependencies then restart but comment all the code blcoks where you need to pip install stuff. or just dont run them alltogether

In [ ]:
#load model
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name =  Config.model_name,
    max_seq_length = Config.max_seq_length,
    dtype = Config.dtype,
    load_in_4bit = Config.load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)


model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.9 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


* If you get accelerate needed error just restart the session because it has already been installed it just hasn't been picked

<a name="Data"></a>
### Data Loading and Preparation


In [ ]:
train = train.reset_index(drop=True)
train['input'] =""
train['input'] = """<bos>[INST] Extract all location names from the given tweet and list them alphabetically, separating each location with a single space., following these guidelines:

Include:
- country, city, county, state, province, or equivalent administrative divisions
- Geo-points (e.g., Eiffel Tower, Statue of Liberty, Mount Kilimanjaro)
- Geo-lines (e.g., Mississippi River, Andes Mountains, Great Wall of China)
- Geo-areas (e.g., New York City, France, Amazon Rainforest, Mediterranean Sea)

Exclude:
- Addresses (e.g., 123 Main St)
- Relative location expressions (e.g., "nearby", "downtown")
- Generic location terms (e.g., "park", "river" without a specific name)

Additional rules:
1. Extract only explicitly named locations falling into the above categories.
2. Present them in alphabetical order.
3. If a location appears multiple times in the tweet with different casing (even if it's just one character different or difference case), include each variation. If the casing is identical, include it only once.
4. Include nested locations separately (e.g., for "Central Park in New York City", list both "Central Park" and "New York City").
5. Include abbreviated forms of locations (e.g., "NYC") and list them where they would appear alphabetically.
6. For locations with qualifiers, include the full phrase (e.g., "Northern France", "Downtown Chicago").
7. Treat multi-word location names as a single entry (e.g., "New York City" is one entry).
8. For locations in hashtags, include the location name without the hash symbol.
9. For ambiguous terms that could be either locations or something else, use context to determine if it's a location.

Tweet: """ + train['text'] + "[/INST]" + train['location'] + tokenizer.eos_token

print(train['input'][0])
display(train.shape, train.head())

<bos>[INST] Extract all location names from the given tweet and list them alphabetically, separating each location with a single space., following these guidelines:

Include:
- country, city, county, state, province, or equivalent administrative divisions
- Geo-points (e.g., Eiffel Tower, Statue of Liberty, Mount Kilimanjaro)
- Geo-lines (e.g., Mississippi River, Andes Mountains, Great Wall of China)
- Geo-areas (e.g., New York City, France, Amazon Rainforest, Mediterranean Sea)

Exclude:
- Addresses (e.g., 123 Main St)
- Relative location expressions (e.g., "nearby", "downtown")
- Generic location terms (e.g., "park", "river" without a specific name)

Additional rules:
1. Extract only explicitly named locations falling into the above categories.
2. Present them in alphabetical order.
3. If a location appears multiple times in the tweet with different casing (even if it's just one character different or difference case), include each variation. If the casing is identical, include it on

(11849, 4)

,tweet_id,text,location,input
0,ID_1001136696589631488,"Flash floods struck a Maryland city on Sunday, washing out streets and tossing cars like bath toys.",Maryland,"<bos>[INST] Extract all location names from the given tweet and list them alphabetically, separating each location with a single space., following these guidelines:\n\nInclude:\n- country, city, county, state, province, or equivalent administrative divisions\n- Geo-points (e.g., Eiffel Tower, Statue of Liberty, Mount Kilimanjaro)\n- Geo-lines (e.g., Mississippi River, Andes Mountains, Great Wall of China)\n- Geo-areas (e.g., New York City, France, Amazon Rainforest, Mediterranean Sea)\n\nExclude:\n- Addresses (e.g., 123 Main St)\n- Relative location expressions (e.g., ""nearby"", ""downtown"")\n- Generic location terms (e.g., ""park"", ""river"" without a specific name)\n\nAdditional rules:\n1. Extract only explicitly named locations falling into the above categories.\n2. Present them in alphabetical order.\n3. If a location appears multiple times in the tweet with different casing (even if it's just one character different or difference case), include each variation. If the casing is identical, include it only once.\n4. Include nested locations separately (e.g., for ""Central Park in New York City"", list both ""Central Park"" and ""New York City"").\n5. Include abbreviated forms of locations (e.g., ""NYC"") and list them where they would appear alphabetically.\n6. For locations with qualifiers, include the full phrase (e.g., ""Northern France"", ""Downtown Chicago"").\n7. Treat multi-word location names as a single entry (e.g., ""New York City"" is one entry).\n8. For locations in hashtags, include the location name without the hash symbol.\n9. For ambiguous terms that could be either locations or something else, use context to determine if it's a location.\n\nTweet: Flash floods struck a Maryland city on Sunday, washing out streets and tossing cars like bath toys.[/INST]Maryland<eos>"
1,ID_1001136950345109504,State of emergency declared for Maryland flooding: via @YouTube,Maryland,"<bos>[INST] Extract all location names from the given tweet and list them alphabetically, separating each location with a single space., following these guidelines:\n\nInclude:\n- country, city, county, state, province, or equivalent administrative divisions\n- Geo-points (e.g., Eiffel Tower, Statue of Liberty, Mount Kilimanjaro)\n- Geo-lines (e.g., Mississippi River, Andes Mountains, Great Wall of China)\n- Geo-areas (e.g., New York City, France, Amazon Rainforest, Mediterranean Sea)\n\nExclude:\n- Addresses (e.g., 123 Main St)\n- Relative location expressions (e.g., ""nearby"", ""downtown"")\n- Generic location terms (e.g., ""park"", ""river"" without a specific name)\n\nAdditional rules:\n1. Extract only explicitly named locations falling into the above categories.\n2. Present them in alphabetical order.\n3. If a location appears multiple times in the tweet with different casing (even if it's just one character different or difference case), include each variation. If the casing is identical, include it only once.\n4. Include nested locations separately (e.g., for ""Central Park in New York City"", list both ""Central Park"" and ""New York City"").\n5. Include abbreviated forms of locations (e.g., ""NYC"") and list them where they would appear alphabetically.\n6. For locations with qualifiers, include the full phrase (e.g., ""Northern France"", ""Downtown Chicago"").\n7. Treat multi-word location names as a single entry (e.g., ""New York City"" is one entry).\n8. For locations in hashtags, include the location name without the hash symbol.\n9. For ambiguous terms that could be either locations or something else, use context to determine if it's a location.\n\nTweet: State of emergency declared for Maryland flooding: via @YouTube[/INST]Maryland<eos>"
2,ID_1001137334056833024,"Other parts of Maryland also saw significant damage from Sundays storms including this Baltimore city neig

In [ ]:

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(train, test_size=0.15, random_state=42)
train_df.shape, val_df.shape

((10071, 4), (1778, 4))

In [ ]:
train_dataset = Dataset.from_pandas(train)
# eval_dataset = Dataset.from_pandas(val_df)

In [ ]:
print(train_dataset['input'][1])

<bos>[INST] Extract all location names from the given tweet and list them alphabetically, separating each location with a single space., following these guidelines:

Include:
- country, city, county, state, province, or equivalent administrative divisions
- Geo-points (e.g., Eiffel Tower, Statue of Liberty, Mount Kilimanjaro)
- Geo-lines (e.g., Mississippi River, Andes Mountains, Great Wall of China)
- Geo-areas (e.g., New York City, France, Amazon Rainforest, Mediterranean Sea)

Exclude:
- Addresses (e.g., 123 Main St)
- Relative location expressions (e.g., "nearby", "downtown")
- Generic location terms (e.g., "park", "river" without a specific name)

Additional rules:
1. Extract only explicitly named locations falling into the above categories.
2. Present them in alphabetical order.
3. If a location appears multiple times in the tweet with different casing (even if it's just one character different or difference case), include each variation. If the casing is identical, include it on

<a name="Train"></a>
#### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    # eval_dataset = eval_dataset,
    dataset_text_field = "input",
    max_seq_length = Config.max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 250,
        num_train_epochs = 2,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 250,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        # max_steps =1500

    ),
)

Map (num_proc=2):   0%|          | 0/11849 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
6.576 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 11,849 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 2,962
 "-____-"     Number of trainable parameters = 54,018,048


Step,Training Loss
250,0.439800
500,0.231100
750,0.228700
1000,0.225900
1250,0.222700
1500,0.221600
1750,0.184800
2000,0.181900
2250,0.182300


Step,Training Loss
250,0.439800
500,0.231100
750,0.228700
1000,0.225900
1250,0.222700
1500,0.221600
1750,0.184800
2000,0.181900
2250,0.182300
2500,0.181600


Step 	Training Loss
250 	0.390300
500 	0.190600
750 	0.185100
1000 	0.182800
1250 	0.179700
1500 	0.177000
1750 	0.144800
2000 	0.143600
2250 	0.142700
2500 	0.143000
2750 	0.141600

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

42435.2112 seconds used for training.
707.25 minutes used for training.
Peak reserved memory = 9.586 GB.
Peak reserved memory for training = 3.01 GB.
Peak reserved memory % of max memory = 64.999 %.
Peak reserved memory for training % of max memory = 20.41 %.


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
# model.save_pretrained("/content/gdrive/MyDrive/Malawi_Finetuning/lora_model_instruct_v2_9_epochs_with_rag_data") # Local saving
model.push_to_hub("Koleshjr/microsoft_learn_2_epochs_gemma_zindi_data_v2", tokenizer, token = userdata.get('HF_TOKEN'), private=True) #push to hub

README.md:   0%|          | 0.00/577 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/216M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Koleshjr/microsoft_learn_2_epochs_gemma_zindi_data_v2


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

In [ ]:
# # #8bit
# model.push_to_hub_gguf("Koleshjr/mistral_7b_v2_8bit_q8_0_1750_steps", tokenizer, token = userdata.get('HF_TOKEN'), private=True)


In [ ]:
# # Save to q4_k_m GGUF
# model.push_to_hub_gguf("Koleshjr/mistral_7b_v2_q4_k_m_1750_steps", tokenizer, quantization_method = "q4_k_m", token = userdata.get('HF_TOKEN'))